In [1]:
import pickle 
import cv2
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import *
from keras.utils import np_utils
from keras.initializers import *
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input, decode_predictions

from art.attacks import FastGradientMethod
from art.attacks import ProjectedGradientDescent
from art.classifiers import KerasClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

/data/aminul/anaconda3/envs/tf_1.4/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/data/aminul/anaconda3/envs/tf_1.4/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/data/aminul/anaconda3/envs/tf_1.4/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/data/aminul/anaconda3/envs/tf_1.4/lib/python3

In [2]:
with open('skin-cancer-dataset', 'rb') as f:
    x_train,x_test,x_val = pickle.load(f)

In [3]:
print(x_train.shape)
print(x_test.shape)
print(x_val.shape)

(8111, 100, 75, 3)
(1002, 100, 75, 3)
(902, 100, 75, 3)


In [4]:
with open('skin-cancer-dataset-label', 'rb') as f:
    y_train,y_test,y_val = pickle.load(f)

In [5]:
print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

(8111, 7)
(1002, 7)
(902, 7)


In [6]:
import cv2
dim=(224,224)
x_new_train=[]
x_new_test=[]
x_new_val=[]

In [7]:
for i in range(8111):
    resized = cv2.resize(x_train[i], dim, interpolation = cv2.INTER_AREA)
    x_new_train.append(resized)
    
for i in range(1002):
    resized = cv2.resize(x_test[i], dim, interpolation = cv2.INTER_AREA)
    x_new_test.append(resized)
    
for i in range(902):
    resized = cv2.resize(x_val[i], dim, interpolation = cv2.INTER_AREA)
    x_new_val.append(resized)

In [8]:
x_new_train = np.array(x_new_train)
x_new_test = np.array(x_new_test)
x_new_val = np.array(x_new_val)

In [9]:
x_new_train = np.concatenate((x_new_train,x_new_val),axis=0)
y_train = np.concatenate((y_train,y_val),axis=0)

In [10]:
print(x_new_train.shape)
print(x_new_test.shape)

(9013, 224, 224, 3)
(1002, 224, 224, 3)


In [11]:
#res = ResNet50(weights=None,input_shape=(224,224,3), classes=7)

In [12]:
#dense = DenseNet121(weights=None,input_shape=(224,224,3), classes=7)
vgg1 = VGG16(weights=None,input_shape=(224,224,3), classes=7)

In [13]:
opt = SGD(lr=0.1,decay=0.002)
vgg1.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
lrr = ReduceLROnPlateau(monitor='val_acc', 
                        patience=3, 
                        verbose=1, 
                        factor=0.5, 
                        min_lr=0.0001)

In [14]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True)  # randomly flip images
datagen.fit(x_new_train)

In [15]:
cb = EarlyStopping(patience=5,verbose =1)
#history = dense.fit(x_new_train,y_train, epochs = 100, batch_size =8, validation_split=0.1,callbacks=[cb,lrr],verbose = 1)
batch_size=8
history = vgg1.fit_generator(
    datagen.flow(x_new_train,y_train, batch_size=8),
    steps_per_epoch=x_train.shape[0] // batch_size,
    epochs=20,
    validation_data=(x_new_val,y_val),
    validation_steps=x_new_val.shape[0] // batch_size
    ,callbacks=[lrr]
)

Epoch 1/20
1013/1013 [==============================] - 195s 193ms/step - loss: 1.1513 - acc: 0.6682 - val_loss: 1.1534 - val_acc: 0.6585
Epoch 2/20
1013/1013 [==============================] - 194s 192ms/step - loss: 1.1288 - acc: 0.6726 - val_loss: 1.1441 - val_acc: 0.6585
Epoch 3/20
1013/1013 [==============================] - 196s 193ms/step - loss: 1.0657 - acc: 0.6685 - val_loss: 1.0120 - val_acc: 0.6585
Epoch 4/20
1013/1013 [==============================] - 196s 194ms/step - loss: 0.9828 - acc: 0.6707 - val_loss: 0.9789 - val_acc: 0.6585

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.05000000074505806.
Epoch 5/20
1013/1013 [==============================] - 194s 191ms/step - loss: 0.9337 - acc: 0.6756 - val_loss: 0.9245 - val_acc: 0.6763
Epoch 6/20
1013/1013 [==============================] - 192s 190ms/step - loss: 0.9066 - acc: 0.6851 - val_loss: 0.8977 - val_acc: 0.6663
Epoch 7/20
1013/1013 [==============================] - 192s 190ms/step - loss: 0.9239 - acc:

In [16]:
print(vgg1.evaluate(x_new_test,y_test))

1002/1002 [==============================] - 7s 7ms/step
[0.7851849673037044, 0.7105788425533119]


## Model Robustness Across MobileNet PGD Images ##

In [19]:
with open('new_adversarial/adversarial_dense_train1', 'rb') as f:
    k = pickle.load(f)
with open('new_adversarial/adversarial_dense_train2', 'rb') as f:
    l = pickle.load(f)
with open('new_adversarial/adversarial_dense_train3', 'rb') as f:
    m = pickle.load(f)
with open('new_adversarial/adversarial_dense_train4', 'rb') as f:
    n = pickle.load(f)

with open('new_adversarial/adversarial_dense_test', 'rb') as f1:
    x_adv_test = pickle.load(f1)

with open('new_adversarial/adversarial_dense_trainY', 'rb') as f:
    y_train_mob = pickle.load(f)
with open('new_adversarial/adversarial_dense_testY', 'rb') as f:
    y_test_mob = pickle.load(f)
    
k = np.array(k)
l = np.array(l)
m = np.array(m)
n = np.array(n)
x_adv_test = np.array(x_adv_test)
y_train_mob = np.array(y_train_mob)
y_test_mob = np.array(y_test_mob )

In [20]:
k = k.reshape(2254,224,224,3)
l = l.reshape(2253,224,224,3)
m = m.reshape(2253,224,224,3)
n = n.reshape(2253,224,224,3)

x_adv_test_mob = x_adv_test.reshape(1002,224,224,3)

y_adv_train_mob = y_train_mob.reshape(9013,7)
y_adv_test_mob = y_test_mob.reshape(1002,7)

In [21]:
print(k.shape)
print(l.shape)
print(m.shape)
print(n.shape)
print(x_adv_test_mob.shape)
print(y_adv_train_mob.shape)
print(y_adv_test_mob.shape)

(2254, 224, 224, 3)
(2253, 224, 224, 3)
(2253, 224, 224, 3)
(2253, 224, 224, 3)
(1002, 224, 224, 3)
(9013, 7)
(1002, 7)


In [22]:
x_adv_train_mob = np.concatenate((k,l,m,n),axis=0)

In [23]:
print("MobileNet Testing Score Before Adversarial Training")
print(vgg1.evaluate(x_adv_test_mob,y_adv_test_mob))

MobileNet Testing Score Before Adversarial Training
1002/1002 [==============================] - 5s 5ms/step
[2.42418676317333, 0.12774451100778675]


## Model Robustness Against VGGNet PGD Adversarial Images ##

In [25]:
with open('new_adversarial/adversarial_vgg_train1', 'rb') as f:
    k = pickle.load(f)
with open('new_adversarial/adversarial_vgg_train2', 'rb') as f:
    l = pickle.load(f)
with open('new_adversarial/adversarial_vgg_train3', 'rb') as f:
    m = pickle.load(f)
with open('new_adversarial/adversarial_vgg_train4', 'rb') as f:
    n = pickle.load(f)

with open('new_adversarial/adversarial_vgg_test', 'rb') as f1:
    x_adv_test = pickle.load(f1)

with open('new_adversarial/adversarial_vgg_trainY', 'rb') as f:
    y_train_vgg = pickle.load(f)
with open('new_adversarial/adversarial_vgg_testY', 'rb') as f:
    y_test_vgg = pickle.load(f)
    
k = np.array(k)
l = np.array(l)
m = np.array(m)
n = np.array(n)
x_adv_test = np.array(x_adv_test)
y_train_vgg = np.array(y_train_vgg)
y_test_vgg = np.array(y_test_vgg )

In [26]:
k = k.reshape(2254,224,224,3)
l = l.reshape(2253,224,224,3)
m = m.reshape(2253,224,224,3)
n = n.reshape(2253,224,224,3)

x_adv_test_vgg = x_adv_test.reshape(1002,224,224,3)

y_adv_train_vgg = y_train_vgg.reshape(9013,7)
y_adv_test_vgg = y_test_vgg.reshape(1002,7)

In [27]:
print(k.shape)
print(l.shape)
print(m.shape)
print(n.shape)
print(x_adv_test.shape)
print(y_train.shape)
print(y_test.shape)

(2254, 224, 224, 3)
(2253, 224, 224, 3)
(2253, 224, 224, 3)
(2253, 224, 224, 3)
(1, 1002, 224, 224, 3)
(9013, 7)
(1002, 7)


In [28]:
x_adv_train_vgg = np.concatenate((k,l,m,n),axis=0)

In [29]:
print("MobileNet Testing Score Before Adversarial Training")
print(vgg1.evaluate(x_adv_test_vgg,y_adv_test_vgg))

MobileNet Testing Score Before Adversarial Training
1002/1002 [==============================] - 5s 5ms/step
[7.394426905466411, 0.08682634730538923]


## Adversarial Training ##

In [31]:
x_adv_train_whole= np.concatenate((x_new_train,x_adv_train_vgg),axis=0)
y_adv_train_whole = np.concatenate((y_train,y_adv_train_vgg),axis=0)

In [32]:
print(x_adv_train_whole.shape)
print(y_adv_train_whole.shape)

(18026, 224, 224, 3)
(18026, 7)


In [33]:
#dense = DenseNet121(weights=None,input_shape=(224,224,3), classes=7)
vgg2 = VGG16(weights=None,input_shape=(224,224,3), classes=7)

In [34]:
opt = SGD(lr=0.1,decay=0.002)
vgg2.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
lrr = ReduceLROnPlateau(monitor='val_acc', 
                        patience=3, 
                        verbose=1, 
                        factor=0.5, 
                        min_lr=0.0001)

In [35]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True)  # randomly flip images
datagen.fit(x_adv_train_whole)

In [36]:
cb = EarlyStopping(patience=5,verbose =1)
batch_size=8
history = vgg2.fit_generator(
    datagen.flow(x_adv_train_whole,y_adv_train_whole, batch_size=8),
    steps_per_epoch=x_adv_train_whole.shape[0] // batch_size,
    epochs=20,
    validation_data=(x_new_val,y_val),
    validation_steps=x_new_val.shape[0] // batch_size
    ,callbacks=[lrr]
)

Epoch 1/20
2253/2253 [==============================] - 428s 190ms/step - loss: 1.1413 - acc: 0.6690 - val_loss: 1.1501 - val_acc: 0.6585
Epoch 2/20
2253/2253 [==============================] - 429s 190ms/step - loss: 1.1302 - acc: 0.6694 - val_loss: 1.1128 - val_acc: 0.6585
Epoch 3/20
2253/2253 [==============================] - 427s 190ms/step - loss: 1.0434 - acc: 0.6692 - val_loss: 0.9793 - val_acc: 0.6585
Epoch 4/20
2253/2253 [==============================] - 427s 189ms/step - loss: 0.9839 - acc: 0.6687 - val_loss: 0.9546 - val_acc: 0.6585

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.05000000074505806.
Epoch 5/20
2253/2253 [==============================] - 429s 190ms/step - loss: 0.9542 - acc: 0.6697 - val_loss: 0.9285 - val_acc: 0.6619
Epoch 6/20
2253/2253 [==============================] - 419s 186ms/step - loss: 0.9369 - acc: 0.6758 - val_loss: 0.9312 - val_acc: 0.6630
Epoch 7/20
2253/2253 [==============================] - 426s 189ms/step - loss: 0.9277 - acc:

In [37]:
print(vgg2.evaluate(x_new_test,y_test))

1002/1002 [==============================] - 6s 6ms/step
[0.8018167934969752, 0.7045908186012162]


## Model Robustness After Training Across MobileNet PGD Images ##

In [38]:
print("MobileNet Testing Score Before Adversarial Training")
print(vgg2.evaluate(x_adv_test_mob,y_adv_test_mob))

MobileNet Testing Score Before Adversarial Training
1002/1002 [==============================] - 5s 5ms/step
[0.9749591038850491, 0.6906187627129926]


## Model Robustness After Training Across  VGG PGD Images ##

In [40]:
print("MobileNet Testing Score Before Adversarial Training")
print(vgg2.evaluate(x_adv_test_vgg,y_adv_test_vgg))

MobileNet Testing Score Before Adversarial Training
1002/1002 [==============================] - 5s 5ms/step
[0.8492311905482096, 0.7105788425533119]


## FGSM Results ##

In [42]:
from art.attacks import FastGradientMethod

In [43]:
min_pixel_value = 0.0
max_pixel_value = 1.0
classifier_1 = KerasClassifier(model= vgg1, clip_values=(min_pixel_value, max_pixel_value), use_logits=False)

In [44]:
eps=0.3
attack_FGSM = FastGradientMethod(classifier=classifier_1)
x_adv_test_FGSM = attack_FGSM.generate(x=x_new_test)

In [46]:
print(vgg1.evaluate(x_adv_test_FGSM,y_test))

1002/1002 [==============================] - 5s 5ms/step
[9.451903059572993, 0.09880239520958084]
